In [1]:
import os
import luigi
import io
import pandas as pd
import numpy as np
from emu.pipeline.download import FileManifest,PatientsLocal,CacheTaskOutput,ExperimentManifest
from emu.pipeline.remote import RemotePatientManifest,RemoteCSV,Patients
# from emu.pipeline.utils import file_ids_by_channel
from emu.luigi.box import BoxTarget, path_to_fid
from emu.pipeline.process import PDilTask
from emu.utils import get_file_manifest,load_patients,Experiment
from emu.auth import jwt
from emu.pdil.raw import download_experiment, BehaviorRaw, timestamps,Game
from emu.pdil.raw import PDilCache as PDC

In [2]:
root_dir = os.path.expanduser('~/.emu/')

In [3]:
pdc = PDC()
exp_files = pdc.data_manifest
exp_files[exp_files.filename.isin(filter(lambda s: s.endswith('taskoutput.mat'),exp_files.filename))]

,filename,id,type,patient_id
3,blockNum_3_computerTT_defect_taskoutput.mat,629615908637,Matfile,1
4,blockNum_4_humanTT_defect_taskoutput.mat,629620274728,Matfile,1
5,blockNum_5_humanTT_coop_taskoutput.mat,629611713944,Matfile,1
6,blockNum_6_computerTT_coop_taskoutput.mat,629611688205,Matfile,1
11,blockNum_1_computerTT_coop_taskoutput.mat,628721484958,Matfile,1
12,blockNum_2_humanTT_coop_taskoutput.mat,628728186668,Matfile,1
13,PRACTICE_blockNum_0 _taskoutput.mat,628698648907,Matfile,1


In [10]:
game = pdc.load_behavior(patient_id=[1])
game.head(20)

DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629615908637, file_name=blockNum_3_computerTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629615908637_blockNum_3_compu_3484a4f2e7   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629620274728, file_name=blockNum_4_humanTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629620274728_blockNum_4_human_73d59337a6   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611713944, file_name=blockNum_5_humanTT_coop_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed s

,points,reaction_time,timing_sumTictocs,timing_wholesession,trial,block,opponent,strategy
0,4,2.271444,17.629951,17.639027,1,1,computer,coop
1,6,2.244568,15.980926,15.999852,2,1,computer,coop
2,1,2.988665,17.021645,17.040217,3,1,computer,coop
3,4,2.751034,16.317291,16.328783,4,1,computer,coop
4,6,1.882266,67.940818,67.967173,5,1,computer,coop
5,4,2.867458,24.961252,24.976686,6,1,computer,coop
6,4,2.365563,16.483597,16.503106,7,1,computer,coop
7,6,0.723958,14.431993,14.448145,8,1,computer,coop
8,6,0.004937,16.430273,16.448066,9,1,computer,coop
9,1,0.329218,60.400191,60.422912,10,1,computer,coop


In [8]:
print(game)

    points  reaction_time  timing_sumTictocs  timing_wholesession  trial  \
0        4       2.271444          17.629951            17.639027      1   
1        6       2.244568          15.980926            15.999852      2   
2        1       2.988665          17.021645            17.040217      3   
3        4       2.751034          16.317291            16.328783      4   
4        6       1.882266          67.940818            67.967173      5   
5        4       2.867458          24.961252            24.976686      6   
6        4       2.365563          16.483597            16.503106      7   
7        6       0.723958          14.431993            14.448145      8   
8        6       0.004937          16.430273            16.448066      9   
9        1       0.329218          60.400191            60.422912     10   
10       4       5.850919          20.940827            20.953631     11   
11       4       1.442510          13.991553            13.999851     12   
12       6  